In [10]:
import pandas as pd
import swifter

info = "/home/jorge/hackaton-cub/info_CUB.csv"
df = pd.read_csv(info, sep=";")

def read_text(path):
    with open(path, "r") as f:
        text = f.read()
    return text

df["text"] = df["text"].swifter.apply(read_text)
# Get lenght words in each text
df["n_words"] = df["text"].swifter.apply(lambda x: len(x.split()))

# Convert labels (str) to int
df["label"] = df["label"].astype("category").cat.codes

labels = df["label"].unique()

print(df["n_words"].describe())
df

Pandas Apply:   0%|          | 0/11788 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11788 [00:00<?, ?it/s]

count    11788.000000
mean       152.287241
std         16.350719
min        109.000000
25%        141.000000
50%        150.000000
75%        162.000000
max        242.000000
Name: n_words, dtype: float64


,n,label,image,text,train,n_words
0,10,0,CUB/images/001.Black_footed_Albatross/Black_Fo...,here we have a bird soaring above the water an...,0,178
1,10,2,CUB/images/003.Sooty_Albatross/Sooty_Albatross...,this is a white bird with grey wings and a dar...,0,142
2,10,3,CUB/images/004.Groove_billed_Ani/Groove_Billed...,"the bird has a black crown, belly, wings, the ...",0,134
3,10,4,CUB/images/005.Crested_Auklet/Crested_Auklet_0...,some sort of web-footed bird with an orange be...,0,175
4,10,5,CUB/images/006.Least_Auklet/Least_Auklet_0010_...,the bird had a white and grey speckled chest w...,0,166
...,...,...,...,...,...,...
11783,143,61,CUB/images/062.Herring_Gull/Herring_Gull_0143_...,the bird is white with black tail feathers tha...,1,155
11784,143,65,CUB/images/066.Western_Gull/Western_Gull_0143_...,"this bird has a white head with yellow bill, i...",1,137
11785,143,117,CUB/images/118.House_Sparrow/House_Sparrow_014...,"this small bird has a grey crown, black beak a...",1,152
11786,144,117,CUB/images/118.House_Sparrow/House_Sparrow_014...,this bird is small with a black and white ches...,0,203


In [11]:
columns = ["text", "label"]
data_train = df[df["train"] == 1][columns]
data_test = df[df["train"] == 0][columns]

In [12]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(data_train).remove_columns(["__index_level_0__"])
test_dataset = Dataset.from_pandas(data_test).remove_columns(["__index_level_0__"])

In [13]:
# Split train in train and validation
train_dataset = train_dataset.train_test_split(test_size=0.1)
eval_dataset = train_dataset["test"]
train_dataset = train_dataset["train"]


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2", padding="max_length", truncation=True)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

loading file vocab.txt from cache at /home/jorge/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/vocab.txt
loading file tokenizer.json from cache at /home/jorge/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/jorge/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/jorge/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/tokenizer_config.json


  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("sentence-transformers/all-mpnet-base-v2", num_labels=len(labels))#.to("cuda")

loading configuration file config.json from cache at /home/jorge/.cache/huggingface/hub/models--sentence-transformers--all-mpnet-base-v2/snapshots/bd44305fd6a1b43c16baf96765e2ecb20bca8e1d/config.json
Model config MPNetConfig {
  "_name_or_path": "sentence-transformers/all-mpnet-base-v2",
  "architectures": [
    "MPNetForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
   

In [16]:
import numpy as np
from transformers import TrainingArguments
from transformers import Trainer

import evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="steps", 
                                  save_strategy="steps",
                                  eval_steps=200,
                                  save_steps=200,
                                  load_best_model_at_end=True,
                                  learning_rate=2e-5, 
                                  per_device_train_batch_size=16, 
                                  per_device_eval_batch_size=16, 
                                  num_train_epochs=4, 
                                  weight_decay=0.01, 
                                  #label_names=[0, 1], 
                                  #metric_for_best_model="eval_f1",
                                  fp16=False)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: text. If text are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5371
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1344
  Number of trainable parameters = 109640264


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
# Predict test set with the model
from sklearn.metrics import classification_report
predictions, labels, _ = trainer.predict(test_dataset)
#predictions = np.argmax(predictions[1], axis=1)
print(classification_report(test_dataset["label"], labels))
